In [1]:
library(RSiena)
library(Matrix)
library(data.table)

# Main webpage with lecture slides and sample code: https://www.stats.ox.ac.uk/~snijders/siena/
# User manual: http://www.stats.ox.ac.uk/~snijders/siena/RSiena_Manual.pdf
# Tutorial: https://www.stats.ox.ac.uk/~snijders/siena/basicRSiena.r


### Data preparation

get_edge_data <- function(data) {
  new <- data[ , c("player_id", "other_id", "count", "round")] 
  new <- new[with(new, order(round, player_id, other_id)), ]
  rownames(new) <- NULL
  return(new)
}

get_wealth_data <- function(data) {
  setDF(data) # Convert to data frame
  new <- data[ , c("player_id", "round", "wealth")] 
  rownames(new) <- NULL
  return(new)
}


get_matrix_per_wave <- function(edge_data, rnd, num_players) {
  
  wave <- edge_data[edge_data$round==rnd, ] 
  wave.copy <- wave[ , 1:3]
  wave.copy <- as.matrix(wave.copy)
  # Create empty adjacency matrix
  adj <- matrix(0, num_players, num_players)
  # Put edge values in desired places
  # "The values of the ties must be 0, 1, orNA(not available = missing); 
  # or 10 or 11 for structurally determined values""
  adj[wave.copy[ , 1:2]] <- 1 #wave.copy[ , 3]
  
  return(adj)
}


get_siena_net <- function(rawdata, t, g) {
  
  edge_data <- get_edge_data(rawdata[(rawdata$treatment==t & rawdata$group==g), ])
  
  # Groups 1, 2, and 29 are missing players and need to shift to have consecutive player ids
  players_in_game <- union(unique(edge_data$player_id), unique(edge_data$other_id))
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    # Find which player is missing - use difference between sets
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    # Decrease player numbers by one if player_id > x to account for missing player x
    edge_data$player_id[edge_data$player_id > x] <- edge_data$player_id[edge_data$player_id > x] - 1
    edge_data$other_id[edge_data$other_id > x] <- edge_data$other_id[edge_data$other_id > x] - 1
  }
  
  # Create adjacency matrix for each round
  num_rounds <- max(edge_data$round)
  matrices <- sapply( 1:num_rounds, function(rnd) get_matrix_per_wave(edge_data, rnd, num_players) )
  giving_data <- array( matrices, dim = c(num_players, num_players, num_rounds) )
  
  return(giving_data)
}


get_siena_endow <- function(rawdata, t, g) {
  # Get endowments in a column ordered by player number
  # No need to worry about missing players since the order is the same after shifting -1
  endow <- unique(rawdata[(rawdata$treatment==t & rawdata$group==g & rawdata$round==1), ])
  endow <- endow[ , c("player_id", "endowment")][order(endow$player_id), ]
  return(endow[, 2])
}


get_siena_wealth <- function(data, t, g) {
  
  # Wealth in previous period
  wealth_data <- data[(data$treatment==t & data$group==g), ]
  wealth_data <- as.data.table(wealth_data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  # Get the sqrt of wealth!!!
  wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  
  # Groups 1, 2, and 29 are missing players and need to shift to have consecutive player ids
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    # Find which player is missing - use difference between sets
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    # Decrease player numbers by one if player_id > x to account for missing player x
    wealth_data$player_id[wealth_data$player_id > x] <- wealth_data$player_id[wealth_data$player_id > x] - 1
  }
  
  wealth_data <- get_wealth_data(wealth_data)
  
  # Create marix with row per player and column per round
  num_rounds <- max(wealth_data$round)
  mat <- matrix(0, num_players, num_rounds)
  # Put wealth values in desired places
  for (i in 1:NROW(wealth_data)) {
    mat[ wealth_data[i, 1], wealth_data[i, 2] ] <- wealth_data[i, 3]
  }
  
  return(mat)
}

get_siena_data <- function(data1, data2, t, g) {
  
  # Create network wave data
  giving <- sienaDependent(get_siena_net(data1, t, g))
  # Add endowment as a constant covariate
  endowment <- coCovar(get_siena_endow(data2, t, g))
  # Add wealth as a time varying covariate
  wealth <- varCovar(get_siena_wealth(data2, t, g))
  
  # Put the variables together for analysis
  print(paste(t, '-', g, sep=""))
  net <- sienaDataCreate( giving, endowment, wealth )
  #print01Report(net, modelname = paste(t, '_', g, sep=""))
  return(net)
  
}

get_siena_effects <- function(net) {
  
  ## RECIPROCITY
  eff <- getEffects(net)
  #effectsDocumentation(eff)
  
  # Control for local bias
  # cycle3 is never significant so we will not include it
  eff <- includeEffects(eff, transTrip)
  
  ### ENDOWMENT
  # Control for the fact that the endowed can give more
  eff <- includeEffects( eff, egoX, interaction1 = "endowment" )
  ## Test for PREFERENTIAL GIVING/AVOIDANCE by endowment
  eff <- includeEffects( eff, altX, interaction1 = "endowment" )
  ## Test for HOMOPHILY by endowment
  # diffX defined  by  the  alter-minus-ego difference of the covariate
  # over all actors to whom i has a tie
  ### MORE SUITABLE BUT fails - Covariance matrix not positive definite
  # simX - 1 if same, 0 if maximum difference 4, 0.5 if difference 2
  # similar results for sameX
  eff <- includeEffects( eff, simX, interaction1 = "endowment" )
  ## Test for INEQUITY AVERSION/RECIPROCITY by endowment
  #eff <- includeEffects( eff, simRecipX, interaction1 = "endowment" )
  
  ### WEALTH
  # Control for the fact that the wealthy may behave
  eff <- includeEffects( eff, egoX, interaction1 = "wealth" )
  ## Test for PREFERENTIAL GIVING/AVOIDANCE by wealth
  eff <- includeEffects( eff, altX, interaction1 = "wealth" )
  ## Test for HOMOPHILY by wealth
  eff <- includeEffects( eff, simX, interaction1 = "wealth" )
  ## Test for INEQUITY AVERSION/RECIPROCITY by endowment
  #eff <- includeEffects( eff, simRecipX, interaction1 = "wealth" )
  
  # ego>alter for covariate(higher), defined by the number of ties where i’s covariate is larger than alter’s, while equality counts for half
  #eff <- includeEffects( eff, sameX, interaction1 = "endowment" )
  #eff <- includeEffects( eff, higher, interaction1 = "endowment" )
  
  return(eff)
  
}


rawdata1 <- read.csv("data/interactions.csv", header=T)
rawdata2 <- read.csv("data/finwealth_from_interactions.csv", header=T)


#for (t in unique(rawdata1$treatment)) {
#  for (g in unique(rawdata1[(rawdata1$treatment==t), ]$group_id)) {
# for (t in c('Endow')) {
#   for (g in c(6)) {

### Run model for each group

# for (t in c('No info')) {
#   for (g in c(29)) {
#     
#     # 1. Get data in siena format
#     net <- get_siena_data(rawdata1, rawdata2, t, g) 
#       
#     # 2. Include effects
#     eff <- get_siena_effects(net)
#     
#     # 3. Define the algorithm
#     alg <- sienaAlgorithmCreate(projname = paste(t, '_', g, sep=""))
# 
#     # 4. Estimate the parameters and save
#     ans <- siena07( alg, data = net, effects = eff)
#     # Save formatted results in a html/tex file
#     siena.table(ans, type="html", file=paste(t, '_', g, '.html'), sig=TRUE)
# 
#   }
# }

### META ANALYSIS

# Define number of effects in model (disregarding rate parameters)
neff <- 9

for (t in unique(rawdata1$treatment)) {
  
  modellist <- list()
  
  for (g in unique(rawdata1[(rawdata1$treatment==t), ]$group_id)) {
    
    # 1. Get data in siena format
    net <- get_siena_data(rawdata1, rawdata2, t, g)
    
    # 2. Include effects
    eff <- get_siena_effects(net)
    
    # 3. Define the algorithm and estimate model
    alg <- sienaAlgorithmCreate(projname = paste(t, '_', g, sep=""))
    (model <- siena07(alg, data = net, effects = eff))
    #siena.table(model, type="html", file=paste(t, '_', g, '.html', sep=""), sig=TRUE)
    # If convergence has to be improved by a further run:
    # if (g %in% c(5)) {
    #   (model <- siena07(alg, data = net, effects = eff, prevAns=model))
    # }
    
    modellist[[paste(t, '_', g, sep="")]] <- model
        
  }
  
  # 4. Combine the groups and do the meta-analysis
  meta <- do.call( siena08, c( modellist, list(projname = paste("meta_", t, sep="")) ) )
  # Divert output to a text file
  summary(meta, file=TRUE, reportEstimates=True)
  
  # 5. Export summary statistics
  
  # Parameter estimates and confidence intervals
  parameters <- t(sapply(1:neff, function(i){c(meta[[i]]$mu.ml, meta[[i]]$mu.ml.se, 
                                             meta[[i]]$mu.confint,
                                             meta[[i]]$sigma.ml, meta[[i]]$sigma.confint,
                                             meta[[i]]$n1)}))
  colnames(parameters) <- c('mu-hat', 'mu-se',
                            'mu-min', 'mu-plus', 'alpha_mu',
                            'sigma-hat', 'sigma-min', 'sigma-plus', 'alpha_sigma', 'N')
  # These are: parameter estimate for population mean; s.e.;
  # confidence interval for population mean (left, right, significance level);
  # estimate for population standard deviation;
  # confidence interval for standard deviation (left, right, significance level);
  # number of groups on which this is based.
  
  # Construct the names for the effects:
  efnames <- names(meta)[1:neff]
  # the first 5 list elements of meta correspond to the tested effects
  # The first 7 letters are redundant and will be dropped:
  efnames <- substring(efnames, 8)
  rownames(parameters) <- efnames
  print(round(parameters, 3))
  # Export
  write.csv(parameters, paste('Siena_output/old_params_', t, '.csv', sep=''))
  
  # All estimates
  write.csv(meta$thetadf, paste('Siena_output/old_thetas_', t, '.csv', sep=''))
  
  # # Extract heterogeneity tests from siena08 output;
  # # these are called Q in Snijders & Baerveldt (2003):
  # hetero <- t(sapply(1:neff, function(i){c(meta[[i]]$Qstat,
  #                                               meta[[i]]$n1-1, meta[[i]]$pttilde)}))
  # colnames(hetero) <- c('Q', 'df', 'pQ')
  # # These are: Q statistic; degrees of freedom; p-value
  # rownames(hetero) <- efnames
  # print(round(hetero, 3))
  # 
  # # Extract overall tests
  # overalls <- t(sapply(1:neff, function(i){c(meta[[i]]$Tsq,
  #                                                 meta[[i]]$n1-1, meta[[i]]$pTsq)}))
  # colnames(overalls) <- c('T^2', 'df', 'pT^2')
  # # These are: Q statistic; degrees of freedom; p-value
  # rownames(overalls) <- efnames
  # print(round(overalls, 3))
  # 
  # # We also present the Fisher combinations:
  # fishers <- t(sapply(1:neff,
  #                     function(i){c(meta[[i]]$cjplus, meta[[i]]$cjminus,
  #                                   meta[[i]]$cjplusp, meta[[i]]$cjminusp,
  #                                   2*meta[[i]]$n1 )}))
  # rownames(fishers) <- efnames
  # colnames(fishers) <- c('Fplus', 'Fminus', 'pplus', 'pminus', 'df')
  # print(round(fishers, 3))
  
}



Warning message:
“no DISPLAY variable so Tk is not available”

[1] "Endow-6"
  effectName          include fix   test  initialValue parm
1 transitive triplets TRUE    FALSE FALSE          0   0   
  effectName    include fix   test  initialValue parm
1 endowment ego TRUE    FALSE FALSE          0   0   
  effectName      include fix   test  initialValue parm
1 endowment alter TRUE    FALSE FALSE          0   0   
  effectName           include fix   test  initialValue parm
1 endowment similarity TRUE    FALSE FALSE          0   0   
  effectName include fix   test  initialValue parm
1 wealth ego TRUE    FALSE FALSE          0   0   
  effectName   include fix   test  initialValue parm
1 wealth alter TRUE    FALSE FALSE          0   0   
  effectName        include fix   test  initialValue parm
1 wealth similarity TRUE    FALSE FALSE          0   0   
siena07 will create an output file Endow_6.txt .


No X11 device available, forcing use of batch mode



Start phase 0 
theta: -1.14  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.17479  0.04956  0.01606 -0.00347  0.00758  0.01508  0.00370 -0.00509  0.03492 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.22241  0.15252  0.03215 -0.00673  0.01875  0.03949  0.00522 -0.00904  0.09837 
ac   0.967   7.503   0.840   0.891   0.924 -10.418   1.397   1.314   1.722 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4

No X11 device available, forcing use of batch mode


...


Start phase 0 
theta: -1.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.154944 -0.000562  0.069177  0.000922  0.011478 -0.018539 -0.000679 -0.000100  0.261256 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.21726 -0.00407  0.13640 -0.00241  0.03849 -0.03793 -0.00181 -0.00109  0.53307 
ac 0.957 0.365 4.990 2.381 1.622 1.125 2.972 2.777 0.339 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress: 15%
the

No X11 device available, forcing use of batch mode



Start phase 0 
theta: -1.01  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.05320  0.05341  0.02364 -0.00103  0.00987 -0.00206  0.00101 -0.00266 -0.05962 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.095348  0.115239  0.035986 -0.002080  0.025746  0.004592  0.000104 -0.004595 -0.044273 
ac 1.116 0.851 1.902 0.966 1.451 2.877 1.219 1.194 0.796 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress

No X11 device available, forcing use of batch mode



Start phase 0 
theta: -0.934  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -0.983161  0.047172  0.029305  0.000922  0.022604 -0.044965 -0.001367  0.000564  0.441431 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.03585  0.10205  0.05187 -0.00086  0.05946 -0.11891 -0.00256 -0.00031  1.10178 
ac 0.913 2.304 0.238 6.274 1.274 1.072 1.403 1.408 1.217 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4